In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NH_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3128618805,NH,"555 Auburn St, Manchester, NH",42.984669,-71.449788,3.301100e+14
1,5946307005,NH,"5 OVERLOOK DR, AMHERST, NH",42.821269,-71.607584,3.301102e+14
2,3345347110,NH,"28 industrial way, ROCHESTER, NH",43.309051,-71.004924,3.301708e+14
3,8543107003,NH,"6 STORRS ST, CONCORD, NH",43.200551,-71.531315,3.301303e+14
4,2415809002,NH,"600 Saint Johnsbury Rd, Littleton, NH",44.317917,-71.825393,3.300996e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3128618805,NH,"555 Auburn St, Manchester, NH",42.984669,-71.449788,3.301100e+14
1,5946307005,NH,"5 OVERLOOK DR, AMHERST, NH",42.821269,-71.607584,3.301102e+14
2,3345347110,NH,"28 industrial way, ROCHESTER, NH",43.309051,-71.004924,3.301708e+14
3,8543107003,NH,"6 STORRS ST, CONCORD, NH",43.200551,-71.531315,3.301303e+14
4,2415809002,NH,"600 Saint Johnsbury Rd, Littleton, NH",44.317917,-71.825393,3.300996e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
213,8312947109,NH,"200 Baker Ave 300, Concord, NH",42.460189,-71.384571,NaN
1381,3985607103,NH,"500 US HWY 1 BYPASS, PORTSMOUTH, NH",43.128510,-70.707675,NaN
2686,8223087000,NH,"1815 Rt 3, COLUMBIA, NH",40.857830,-74.179704,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('33')] #NH
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/3 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

3


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15311/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
213,8312947109,NH,"200 Baker Ave 300, Concord, NH",42.460189,-71.384571,[]
1381,3985607103,NH,"500 US HWY 1 BYPASS, PORTSMOUTH, NH",43.128510,-70.707675,[]
2686,8223087000,NH,"1815 Rt 3, COLUMBIA, NH",40.857830,-74.179704,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15311/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
213,8312947109,NH,"200 Baker Ave 300, Concord, NH",42.460189,-71.384571,NaN
1381,3985607103,NH,"500 US HWY 1 BYPASS, PORTSMOUTH, NH",43.128510,-70.707675,NaN
2686,8223087000,NH,"1815 Rt 3, COLUMBIA, NH",40.857830,-74.179704,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
213,8312947109,NH,"200 Baker Ave 300, Concord, NH",42.460189,-71.384571,NaN
1381,3985607103,NH,"500 US HWY 1 BYPASS, PORTSMOUTH, NH",43.128510,-70.707675,NaN
2686,8223087000,NH,"1815 Rt 3, COLUMBIA, NH",40.857830,-74.179704,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
213,8312947109,NaN
1381,3985607103,NaN
2686,8223087000,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NH_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,553263,3128618805,NH,"555 Auburn St, Manchester, NH",42.984669,-71.449788,3.301100e+14
1,553264,5946307005,NH,"5 OVERLOOK DR, AMHERST, NH",42.821269,-71.607584,3.301102e+14
2,553265,3345347110,NH,"28 industrial way, ROCHESTER, NH",43.309051,-71.004924,3.301708e+14
3,553266,8543107003,NH,"6 STORRS ST, CONCORD, NH",43.200551,-71.531315,3.301303e+14
4,553267,2415809002,NH,"600 Saint Johnsbury Rd, Littleton, NH",44.317917,-71.825393,3.300996e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,553263,3128618805,NH,"555 Auburn St, Manchester, NH",42.984669,-71.449788,3.301100e+14,NaN
1,553264,5946307005,NH,"5 OVERLOOK DR, AMHERST, NH",42.821269,-71.607584,3.301102e+14,NaN
2,553265,3345347110,NH,"28 industrial way, ROCHESTER, NH",43.309051,-71.004924,3.301708e+14,NaN
3,553266,8543107003,NH,"6 STORRS ST, CONCORD, NH",43.200551,-71.531315,3.301303e+14,NaN
4,553267,2415809002,NH,"600 Saint Johnsbury Rd, Littleton, NH",44.317917,-71.825393,3.300996e+14,NaN
...,...,...,...,...,...,...,...,...
5064,558327,8327357003,NH,"15 INDUSTRIAL PARK DR, BERLIN, NH",44.510365,-71.156578,3.300795e+14,NaN
5065,558328,8453917202,NH,"453 CENTRAL AVENUE #107, DOVER, NH",43.197461,-70.873775,3.301708e+14,NaN
5066,558329,8581768304,NH,"114 Mast Rd, Lee, NH",43.135207,-70.988428,3.301708e+14,NaN
5067,558330,9240498803,NH,"182 W Main St, Conway, NH",43.976081,-71.132761,3.300396e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,553263,3128618805,NH,"555 Auburn St, Manchester, NH",42.984669,-71.449788,3.301100e+14,NaN,3.301100e+14
1,553264,5946307005,NH,"5 OVERLOOK DR, AMHERST, NH",42.821269,-71.607584,3.301102e+14,NaN,3.301102e+14
2,553265,3345347110,NH,"28 industrial way, ROCHESTER, NH",43.309051,-71.004924,3.301708e+14,NaN,3.301708e+14
3,553266,8543107003,NH,"6 STORRS ST, CONCORD, NH",43.200551,-71.531315,3.301303e+14,NaN,3.301303e+14
4,553267,2415809002,NH,"600 Saint Johnsbury Rd, Littleton, NH",44.317917,-71.825393,3.300996e+14,NaN,3.300996e+14


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3128618805,NH,"555 Auburn St, Manchester, NH",42.984669,-71.449788,3.301100e+14
1,5946307005,NH,"5 OVERLOOK DR, AMHERST, NH",42.821269,-71.607584,3.301102e+14
2,3345347110,NH,"28 industrial way, ROCHESTER, NH",43.309051,-71.004924,3.301708e+14
3,8543107003,NH,"6 STORRS ST, CONCORD, NH",43.200551,-71.531315,3.301303e+14
4,2415809002,NH,"600 Saint Johnsbury Rd, Littleton, NH",44.317917,-71.825393,3.300996e+14
...,...,...,...,...,...,...
5064,8327357003,NH,"15 INDUSTRIAL PARK DR, BERLIN, NH",44.510365,-71.156578,3.300795e+14
5065,8453917202,NH,"453 CENTRAL AVENUE #107, DOVER, NH",43.197461,-70.873775,3.301708e+14
5066,8581768304,NH,"114 Mast Rd, Lee, NH",43.135207,-70.988428,3.301708e+14
5067,9240498803,NH,"182 W Main St, Conway, NH",43.976081,-71.132761,3.300396e+14


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
213,8312947109,NH,"200 Baker Ave 300, Concord, NH",42.460189,-71.384571,NaN
1381,3985607103,NH,"500 US HWY 1 BYPASS, PORTSMOUTH, NH",43.128510,-70.707675,NaN
2686,8223087000,NH,"1815 Rt 3, COLUMBIA, NH",40.857830,-74.179704,NaN


Convert to File

In [19]:
fips_NH = fips_merge_drop

In [20]:
fips_NH.to_csv("../../../data/state_data/geo/geo_fips/23/NH_fips_scraped.csv") 